In [ ]:
import pandas as pd     
import numpy as np 
from tabulate import tabulate
import holidays 
from sportsipy.fb.schedule import Schedule


In [ ]:
master = pd.read_parquet("./processed_data/2_non_null_master.parquet")
print(tabulate(master, headers="keys", tablefmt="psql"))


# Translation Tables & Handling string values
En aquest petit apartat generarem taules de traduccio de enumeracions (Haurem també de crear aquestes enumeracions, per tal de guardar espai a les dades i  també per si en el futur volem que el model de machine learning les faci servir, no podrá si es tracta de strings). Primerament identificarem quines columnes d'strings mereixen ser tractades: 

- centro_carga
- id_cerveza & desc_cerveza
- id_cliente & desc_cliente
- id_vehiculo
- id_proveedor, id_fiscal_proveedor & desc_proveedor

En la taula mestra, encomptes de deixar els camps rellevants en forma de enumeració, farem us dels valors categòrics de Pandas.

In [ ]:
#Get the translation dataframe
enum, centro_carga_categories = pd.factorize(master["centro_carga"])
centro_carga_df = pd.DataFrame({"centro_carga": centro_carga_categories, "centro_carga_enumeration": range(len(centro_carga_categories))})

#update the master 
master["centro_carga"] = master["centro_carga"].astype("category")


In [ ]:
enumeration, id_cerveza_categories = pd.factorize(master["id_cerveza"])
#a join is needed here
conversion = master[["desc_cerveza", "id_cerveza"]]
cerveza_df = pd.DataFrame({"desc_cerveza": conversion["desc_cerveza"].unique(), "id_cerveza": conversion["id_cerveza"].unique(), "cerveza_enumeration": range(len(id_cerveza_categories)) })
print(cerveza_df)

master[["id_cerveza", "desc_cerveza"]] = master[["id_cerveza", "desc_cerveza"]].astype("category")

In [ ]:
enumeration, id_proveedor_categories = pd.factorize(master["id_proveedor"])

conversion = master[["id_proveedor", "id_fiscal_proveedor", "empresa_proveedor"]]
proveedor_df = pd.DataFrame({"id_proveedor": conversion["id_proveedor"].unique(), "id_fiscal_proveedor": conversion["id_fiscal_proveedor"].unique(), "proveedor_enumeration": range(len(id_proveedor_categories))})
subproveedor_df = master[["id_proveedor", "empresa_proveedor"]]
subproveedor_df = subproveedor_df.drop_duplicates()
subproveedor_df["empresa_proveedor_enumeration"] = range(len(subproveedor_df["id_proveedor"]))

#convert these columns into categorical instead of string
master[["id_proveedor", "id_fiscal_proveedor", "empresa_proveedor"]] = master[["id_proveedor", "id_fiscal_proveedor", "empresa_proveedor"]].astype("category")
print(proveedor_df)
print(subproveedor_df)


In [ ]:
conversion = master[["id_cliente", "desc_cliente"]].drop_duplicates()
conversion["cliente_enumeration"] = range(len(conversion["id_cliente"]))
cliente_df = conversion
print(cliente_df)

master[["id_cliente", "desc_cliente"]] = master[ ["id_cliente", "desc_cliente"] ].astype("category")
master["id_cliente_enum"] = master["id_cliente"].astype('category').cat.codes


In [ ]:
vehiculo_df = pd.DataFrame(master["id_vehiculo"].unique())
vehiculo_df["vehiculo_enumeration"] = range(len(master["id_vehiculo"].unique()))

print(vehiculo_df)
master["id_vehiculo"] = master["id_vehiculo"].astype("category")


## Festividades & Partidos Liga

In [ ]:
from config import years 

def get_spain_holidays(years):
    spain_holidays = holidays.Spain(years=years)
    holidays_data = [(pd.to_datetime(date), name) for date, name in sorted(spain_holidays.items())]

    holidays_df = pd.DataFrame(holidays_data, columns=["fecha", "festividad"])
    return holidays_df


festividades = get_spain_holidays(years)
festividades.dtypes

In [ ]:
"""
from config import teams 

def get_football_events(teams):
    all_events_df = pd.DataFrame()

    for team in teams:
        schedule = sportsipy.fb.Schedule(team)
        events_data = []

        for game in schedule:
            event_data = {
                'Date': game.datetime,
                'Home_Team': game.home_team,
                'Away_Team': game.away_team,
                'Goals_Scored': game.goals_scored,
                'Goals_Allowed': game.goals_allowed,
                'Result': game.result,
                'Venue': game.location,
            }
            events_data.append(event_data)

        yearly_events_df = pd.DataFrame(events_data)
        all_events_df = pd.concat([all_events_df, yearly_events_df], ignore_index=True)

    return all_events_df

# Example usage for La Liga events for the specified teams
la_liga_schedule = get_football_events(teams)
print(la_liga_schedule)
"""

In [ ]:
master["fecha"] = pd.to_datetime(master["fecha_pedido"])
master["pedido_festivo"] = master["fecha"].isin(festividades["fecha"])
master.drop("fecha", axis=1, inplace=True)
len(master[master["pedido_festivo"] == False])
#len(master[master["pedido_festivo"] == True])

In [ ]:
#Save categorical master 
master.to_parquet("./processed_data/3_categorical_master.parquet",index=True)
#Save traduction tables
centro_carga_df.to_parquet("./processed_data/3_centros_de_carga.parquet", index=True)
cerveza_df.to_parquet("./processed_data/3_cervezas.parquet", index=True)
proveedor_df.to_parquet("./processed_data/3_proveedores.parquet", index=True)
subproveedor_df.to_parquet("./processed_data/3_subproveedores.parquet", index=True)
cliente_df.to_parquet("./processed_data/3_clientes.parquet", index=True)
vehiculo_df.to_parquet("./processed_data/3_vehiculos.parquet", index=True)
